In [ ]:
import pandas as pd
import os
import json
from openai import OpenAI
from tqdm import tqdm  # For the progress bar
import time

In [ ]:
# --- 1. CONFIGURATION: SET YOUR VARIABLES HERE ---

# Set your API key as an environment variable for security
# In your terminal, run: export OPENAI_API_KEY='sk-...'
try:
    client = OpenAI(api_key="sk-proj-3yvwUcXzg8sBGfLeOlGCGRV6VszL1PPU-Qz9KypF4919ksA")
except KeyError:
    print("Error: OPENAI_API_KEY environment variable not set.")
    print("Please set it by running: export OPENAI_API_KEY='sk-...'")
    exit()

# The model to use. 'gpt-4o' is smart. 'gpt-3.5-turbo' is cheaper/faster.
MODEL_TO_USE = "gpt-4o"

# Define your file paths product_name
INPUT_FILE = "/content/Product List - Miri product master product list.csv"
OUTPUT_FILE = "ProductList_master_cleaned.csv"
COLUMN_TO_CLEAN = "ProductName" # The *exact* name of the column in your CSV

# How many rows to process before saving (your 30-40 chunk)
SAVE_CHUNK_SIZE = 50

# --- 2. THE "BRAIN": THE AI PROMPT AND FUNCTION ---
# This is the MOST IMPORTANT part.
# I have added your new, specific examples to make the AI smarter.

SYSTEM_PROMPT = """
You are an expert data extraction and cleaning bot. You will be given a single product string.
Your task is to extract the 'main_product', 'product_type' and 'SKU' from the string.

For each input `product_name`, return this JSON:

{
  "main_product": "<brand or main name>",
  "product_type": "<variant, flavor, or type if present >",
  "SKU": "<quantity/unit if present>",
}

RULES:
- 'main_product': The main_product name, usually the first word(s) or analise by your self sometime two words too. Represents brand, product line, or main identity.
- 'product_type': The middle part of the description (e.g., 'soap', 'Rose', 'Powder', 'Cutter','butter' , 'masala').
- 'SKU': The measurement OR MRP (e.g., '100gm', '1L', '5kg', 'RS 15', '15 RS', 'Rs.1', '10', '40w' ,'pcs' ,'ml').
- 'SPELLING CORRECTION': You MUST correct any spelling errors found in the input string and normalize casing (e.g., "Mashroom" -> "Mushroom", "Chily" -> "Chilli", "Agarwatti" -> "Agarbatti"). This is critical.
- 'SINGLE WORD PRODUCTS': If the input is a single word (e.g., "Egg", "Mushroom", "Paneer"), it MUST be placed in 'main_product', and both 'product_type' and 'SKU' MUST be empty strings "".
- If a value is not found, you MUST return an empty string "" for that key.
- Try to analyze the string yourself to best determine the main_product, product_type, SKU .

--- GENERAL ANALYSIS PRINCIPLE (THEORY) ---
To solve this, you should analyze the string in three logical steps:
1.  **Find the SKU:** First, look for the 'SKU' (price, weight, or count,ETC). This is the easiest part to identify. It almost always contains numbers (like '100gm', '50g', '10 RS', 'Rs.1', '6PCS', '40 w', 'MRP 105') and is almost always at the *end* of the string.
2.  **Find the main_product:** Second, look for the 'main_product'. This is almost always at the *beginning* of the string FIRST one two words.
  'SPELLING CORRECTION': You MUST correct any spelling errors found in the input string and normalize casing (e.g., "Mashroom" -> "Mushroom", "Chily" -> "Chilli", "Agarwatti" -> "Agarbatti").
3.  **Find the Product Type:** Third, the 'product_type' is whatever descriptive text is left in the *middle* *after* you have identified the main_product (from the front) and the SKU (from the back).
   'SPELLING CORRECTION': You MUST correct any spelling errors found in the input string and normalize casing.
4. **Spelling Correction**
   Always correct common spelling mistakes and normalize casing (Agarwatti → Agarbatti, Mashroom → Mushroom, etc.)

--- Special Cases for this Theory ---
- If the string has no middle text (like "Dyna -125 gm."), the 'product_type' is "".
- If the string has no SKU at the end (like "Breeze Sandle Sparsh"), the 'SKU' is "".
- If the string has no obvious main_product at the beginning (like "Spicy Chilli"), the 'main_product' is "".

--- DETAILED EXAMPLES (PRACTICE) ---
-- if for example organic soap then main_product also organic soap product_type soap also because we need proper detail from main_product and organic mean what not clear --
-- look think by yourself too Kurti     With Plazzo or other related this type is main_product in general term so why in product_type think like humen too --
-- for each row think by yourself too with these all prompt like humen --

Input: "Doctor soap -100 gm"
Output: {"main_product": "Doctor Soap", "product_type": "Soap", "SKU": "100 gm"}

Input: "Dyna Rose -50 gm"
Output: {"main_product": "Dyna", "product_type": "Rose", "SKU": "50 gm"}

Input: "Honey And Turmeric Soap."
Output: {"main_product": "Honey And Turmeric Soap", "product_type": "Soap", "SKU": ""}

Input: "Spicy Chilli"
Output: {"main_product": "Spicy Chilli", "product_type": "", "SKU": ""}

Input: "Egg"
Output: {"main_product": "Egg", "product_type": "", "SKU": ""}

Input: "Paneer"
Output: {"main_product": "Paneer", "product_type": "", "SKU": ""}

Input: "Banana Chips 50g"
Output: {"main_product": "Banana Chips", "product_type": "Chips", "SKU": "50g"}

Input: "Charcoal        Agarbatti"
Output: {"main_product": "Charcoal Agarbatti", "product_type": "Agarbatti", "SKU": ""}

Input: "Oyester Mashroom"
Output: {"main_product": "Oyster Mushroom", "product_type": "", "SKU": ""}

Input: "Cushion Cover"
Output: {"main_product": "Cushion Cover", "product_type": "Cover", "SKU": ""}

Input: "Breeze Sandle Sparsh"
Output: {"main_product": "Breeze", "product_type": "Sandal Sparsh", "SKU": ""}

Input: "Lays Classic 50g"
Output: {"main_product": "Lays", "product_type": "Classic", "SKU": "50g"}

Input: "Parle-G Gold Biscuits -10 RS"
Output: {"main_product": "Parle-G", "product_type": "Gold Biscuits", "SKU": "10 RS"}

Input: "unbranded loose sugar 1kg"
Output: {"main_product": "unbranded", "product_type": "loose sugar", "SKU": "1kg"}

Input: "Gazak - Cutter"
Output: {"main_product": "Gazak", "product_type": "Cutter", "SKU": ""}

Input: "Super Garam Masala Jeet - Rs.1"
Output: {"main_product": "Super Garam Masala", "product_type": "Jeet", "SKU": "Rs.1"}

Input: "Phynail"
Output: {"main_product": "Phenyl", "product_type": "", "SKU": ""}
note - see in phynail i corrected spelling too from wrong phynail to phenyl

Input: "Airtel - 10"
Output: {"main_product": "Airtel", "product_type": "", "SKU": "10"}

Input: "P BINGO CHIPS MRP 105"
Output: {"main_product": "P BINGO", "product_type": "CHIPS", "SKU": "MRP 105"}

Input: "CEAM BON(6PCS)RKS"
Output: {"main_product": "CEAM BON", "product_type": "", "SKU": "6PCS"}

Input: "Flattened Rice"
Output: {"main_product": "Flattened Rice", "product_type": "Rice", "SKU": ""}

"""
def clean_product_name(product_string):
    """
    Sends a single product string to the AI and gets a structured JSON response.
    """
    # Handle empty or bad data *before* sending to API
    if not isinstance(product_string, str) or product_string.strip() == "":
        return {"main_product": "", "product_type": "", "SKU": ""}

    try:
        response = client.chat.completions.create(
            model=MODEL_TO_USE,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": product_string}
            ],
            # This 'json_object' mode FORCES the AI to return valid JSON.
            # This is critical for production code.
            response_format={"type": "json_object"},
            temperature=0 # Set to 0 for consistent, non-creative answers
        )

        # Extract the JSON string from the response
        result_json = response.choices[0].message.content

        # Convert the JSON string into a Python dictionary
        data = json.loads(result_json)
        return data

    except Exception as e:
        # Handle API errors, rate limits, etc.
        print(f"\n--- ERROR ---")
        print(f"Error processing string: '{product_string}'")
        print(f"Error details: {e}")
        print(f"Will retry after 5 seconds...")
        print(f"-------------")
        time.sleep(5)
        # You could also return a default error structure
        return {"main_product": "API_ERROR", "product_type": str(e), "SKU": product_string}


# --- 3. THE MAIN SCRIPT: LOOP, PROCESS, AND SAVE ---

def main():
    print(f"Starting data cleaning process for '{INPUT_FILE}'...")

    # Load the entire messy CSV
    try:
        df = pd.read_csv(INPUT_FILE)
    except FileNotFoundError:
        print(f"Error: Input file '{INPUT_FILE}' not found.")
        return
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return

    print(f"Loaded {len(df)} total rows.")

    # Define the new columns we will be adding
    new_columns = ['main_product', 'product_type', 'SKU']

    # --- Resume Logic ---
    # Check if the output file already exists to resume
    start_row = 0
    if os.path.exists(OUTPUT_FILE):
        try:
            df_cleaned_check = pd.read_csv(OUTPUT_FILE)
            start_row = len(df_cleaned_check)
            print(f"Output file found. Resuming from row {start_row}.")
        except pd.errors.EmptyDataError:
            print("Output file is empty. Starting from scratch.")
            # We'll write headers later
        except Exception as e:
            print(f"Error reading output file: {e}. Starting from scratch.")

    if start_row == 0:
        print("No output file found (or it was empty). Starting from scratch.")
        # Create the file and write headers
        # It has all original columns + our 3 new ones
        header_df = pd.DataFrame(columns=list(df.columns) + new_columns)
        header_df.to_csv(OUTPUT_FILE, index=False)


    # A list to hold our chunk of results
    results_chunk = []

    # Get the rows that still need to be processed
    df_to_process = df.iloc[start_row:]

    if len(df_to_process) == 0:
        print("All rows have already been processed. Exiting.")
        return

    print(f"Processing {len(df_to_process)} remaining rows...")

    # Use tqdm for a progress bar
    # We loop one row at a time (index, row)
    for index, row in tqdm(df_to_process.iterrows(), total=len(df_to_process)):

        try:
            original_name = row[COLUMN_TO_CLEAN]
        except KeyError:
            print(f"\nError: Column '{COLUMN_TO_CLEAN}' not found in your CSV.")
            print(f"Available columns are: {list(df.columns)}")
            return

        # 1. Call the AI function
        cleaned_data = clean_product_name(original_name)

        # 2. Combine original row data with new cleaned data
        new_row_data = row.to_dict()
        new_row_data['main_product'] = cleaned_data.get('main_product', '')
        new_row_data['product_type'] = cleaned_data.get('product_type', '')
        new_row_data['SKU'] = cleaned_data.get('SKU', '')

        results_chunk.append(new_row_data)

        # 3. Save to file in chunks (this is your 40-row chunk logic)
        if len(results_chunk) >= SAVE_CHUNK_SIZE:
            # Convert list of dicts to a DataFrame
            df_chunk = pd.DataFrame(results_chunk)

            # Append to the CSV file *without* the header
            df_chunk.to_csv(OUTPUT_FILE, mode='a', header=False, index=False)

            # Clear the list for the next chunk
            results_chunk = []

    # 4. Save any remaining rows after the loop finishes
    if len(results_chunk) > 0:
        df_chunk = pd.DataFrame(results_chunk)
        df_chunk.to_csv(OUTPUT_FILE, mode='a', header=False, index=False)
        print(f"\nSaved final {len(results_chunk)} rows.")

    print(f"\nAll processing complete! Cleaned data saved to '{OUTPUT_FILE}'.")


# Run the main function when the script is executed
if __name__ == "__main__":
    main()

Starting data cleaning process for '/content/Product List - Miri product master product list.csv'...
Loaded 1289 total rows.
No output file found (or it was empty). Starting from scratch.
Processing 1289 remaining rows...


  4%|▍         | 52/1289 [00:41<16:17,  1.26it/s]


KeyboardInterrupt: 